In [1]:
from bellek.utils import set_seed
import numpy as np
from datasets import load_dataset, DatasetDict, Dataset

set_seed(89)

## N-hop variants

In [2]:
def publish_nhop_variant(path: str, config_name: str, n_hop: int):
    dsd = load_dataset(path, config_name)
    target_dsd = DatasetDict()
    for split, ds in dsd.items():
        target_dsd[split] = ds.filter(lambda example: len(example['question_decomposition']) == n_hop)
    target_dsd.push_to_hub(f"{path}-{n_hop}hop", config_name=config_name)

In [3]:
# publish_nhop_variant("bdsaglam/musique", "default", 2)
# publish_nhop_variant("bdsaglam/musique", "answerable", 2)

## Mini version with equal distribution of number of hops

In [4]:
# a function that samples from the dataset with equal distribution of n_hops
def sample_evenly(dataset, n_samples):
    dataset = dataset.map(lambda x: {'n_hops': len(x['question_decomposition'])})
    n_hops = np.unique(dataset['n_hops'])
    samples_per_hop = n_samples // len(n_hops)
    for hop in n_hops:
        hop_samples = dataset.filter(lambda x: x['n_hops'] == hop).shuffle().select(range(samples_per_hop))
        yield from hop_samples

In [7]:
def publish_mini_variant(path: str, config_name: str, n_samples: int):
    dsd = load_dataset(path, config_name)
    target_dsd = DatasetDict()
    for split, ds in dsd.items():
        target_dsd[split] = Dataset.from_list(list(sample_evenly(ds, n_samples)))
    target_dsd.push_to_hub(f"{path}-mini", config_name=config_name)

In [ ]:
publish_mini_variant("bdsaglam/musique", "default", 300)
publish_mini_variant("bdsaglam/musique", "answerable", 300)

## The subset used in my thesis

In [8]:
from bellek.musique.constants import ABLATION_RECORD_IDS

def publish_thesis_variant(path: str, config_name: str):
    dsd = load_dataset(path, config_name)
    target_dsd = DatasetDict()
    for split, ds in dsd.items():
        target_dsd[split] = ds.filter(lambda x: x['id'] in ABLATION_RECORD_IDS)
    target_dsd.push_to_hub(f"{path}-thesis", config_name=config_name)

In [ ]:
publish_thesis_variant("bdsaglam/musique", "answerable")